<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code implements a binomial model to price American options, considering early exercise features. Using parameters like spot price, strike price, risk-free rate, volatility, time to expiry, and the number of steps, the model constructs a price tree. It then calculates the option value at each node, considering both holding and exercising the option. This approach is practical for valuing American-style options, which can be exercised at any time before expiration.

In [ ]:
import numpy as np

Define a function to price American options using the binomial tree model

In [ ]:
def american_option_pricer(spot, strike, rate, vol, expiry, steps, option_type):
    """Price an American option using binomial model
    
    Parameters
    ----------
    spot : float
        Current spot price of the underlying asset
    strike : float
        Strike price of the option
    rate : float
        Risk-free interest rate
    vol : float
        Volatility of the underlying asset
    expiry : float
        Time to expiry in years
    steps : int
        Number of steps in the binomial tree
    option_type : str
        Type of the option ('call' or 'put')
    
    Returns
    -------
    float
        Estimated price of the American option
    
    Notes
    -----
    This function constructs a binomial tree to 
    estimate the price of an American option. It 
    accounts for early exercise features by comparing 
    the value of holding versus exercising the option 
    at each node.
    """

    # Calculate the time interval and the up and down factors

    dt = expiry / steps
    u = np.exp(vol * np.sqrt(dt))
    d = 1 / u

    # Calculate the risk-neutral probability

    p = (np.exp(rate * dt) - d) / (u - d)

    # Create the binomial price tree

    price_tree = np.zeros((steps + 1, steps + 1))
    for i in range(steps + 1):
        price_tree[i, -1] = spot * (u ** (steps - i)) * (d**i)

    # Calculate the option value at each node

    option_tree = np.zeros_like(price_tree)
    if option_type.lower() == "call":
        option_tree[:, -1] = np.maximum(price_tree[:, -1] - strike, 0)
    elif option_type.lower() == "put":
        option_tree[:, -1] = np.maximum(strike - price_tree[:, -1], 0)
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    # Traverse the tree backward to find the option price today

    for t in range(steps - 1, -1, -1):
        for i in range(t + 1):
            exercise = 0
            if option_type.lower() == "call":
                exercise = price_tree[i, t] - strike
            elif option_type.lower() == "put":
                exercise = strike - price_tree[i, t]

            hold = np.exp(-rate * dt) * (
                p * option_tree[i, t + 1] + (1 - p) * option_tree[i + 1, t + 1]
            )
            option_tree[i, t] = np.maximum(exercise, hold)

    return option_tree[0, 0]

Estimate the price of an American Call option using the defined binomial model parameters

In [ ]:
option_price = american_option_pricer(
    spot=55.0,
    strike=50.0,
    rate=0.05,
    vol=0.3,
    expiry=1.0,
    steps=100,
    option_type="Call",
)

Print the estimated price of the American Call option

In [ ]:
print(
    f"The estimated price of the American Call option is: {option_price:.2f}"
)

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.